In [80]:
import pdfplumber
import regex as re
import pyawk # so far not used

In [64]:
pdf_file = open('../data/0683-21.pdf', 'rb')
### imported pyawk but haven't set it up yet

### problems:
### 0483-21.pdf has 2 empty pages before the entwurf starts :((((

### Goal:
# input: raw PDF of antwurf
# output: list of (law title, change request text) pairs

# next wishful steps:
# Analyse entwurf patterns, common, less common etc.
# API to grab law text using that damn title
# API to grab entwurfs

In [65]:
with pdfplumber.open(pdf_file) as pdf:
    text_of_change = [page.extract_text() for page in pdf.pages]

In [77]:
#quirky mehtod to find the first page where the Entwurf starts in the PDF
#it picks the first page after an empty page, which is identrified as a None in the list, or as a string of length 1 :(((
for pagenum, page in enumerate(text_of_change):
    if page is None or len(page)==1:
        first_page=pagenum+1
        break

last_page=len(text_of_change)

#quirky mehtod to find the last page where the Entwurf ends in the PDF
#it picks the last page before a page that has the word "Begründung" in it
for page in range(first_page,last_page):
    if len(text_of_change[page].split('Begründung')) > 1:
        last_page=page-1
        break

# uncomment next line to manually check if the fist and last page were identified correctly
print(first_page,last_page)

4 5


In [78]:
# goes over the entwurf pages, and makes a list of all the 2 lines of text after the patten "Artikel [number] \n", 
# also in case there is more than one change in a page

title_change_of_this_law = []
for page in range(first_page,last_page+1):
    has_artikel = len(re.split('Artikel [0-9]+ +\n',text_of_change[page]))  # the len() of the split is 1 is there are no artikles, 2, if there is 1, and 
                                                                            # increases by 2 (odds) for every additional artikel
    #print(page,has_artikel) # used for debugging only, can be removed
    if has_artikel >= 2:
        for artikel in range(1,has_artikel,2): 
            title_change_of_this_law.extend(re.search('.+\n.+\n',re.split('Artikel [0-9]+ +\n',text_of_change[page])[artikel]).captures())

# the logic for this funciton is as follows:
# in each page, each time the pattern "Artikel [number] \n" appears, 
# note that number regexed any number of digits, this could be limited in the future to only 1-2 or 3?! which is more reasonable,
# we split the string and take the part that is after the split (if there is only one split that's 1, if there is more we loop on the odd numbers to 
# always take the part after the split), we then use re.search to take the first two scentences using a pattern that take any number of charcters 
# before a new line (twice). Since re.search() returns an object we use .capture to to grab the match of that pattern from the returned object.
# since that that match is a list with one string in it, we use .extend() so that our final input is one flat list with all the strings we recovered.
# huzzah.

In [79]:
title_change_of_this_law

['Änderung des Bürgerlichen Gesetzbuchs\nDas Bürgerliche Gesetzbuch in der Fassung der Bekanntmachung vom 2. Januar 2002 (BGBl. \n',
 'Änderung des Einführungsgesetzes zum Bürgerlichen Gesetzbuche\nDem Artikel 229 des Einführungsgesetzes zum Bürgerlichen Gesetzbuche in der Fassung der \n']

In [81]:
# this is an unfinished code chunk
# the following 2 lines only remove the last line break from teh end of the 2nd scentence
[change_title.rstrip() for change_title in title_change_of_this_law]

# the goal is to add a simple logic that capture a second line when it is relevant, e.g. if it start with "und " or if it is no longer than 3 words.
# this won't catch all cases, but it's a start
# more logic to consider: don't take a 2nd line that starts with "§" or "In"

# strip the begining from "Änderung des" or "Weitere Änderung des"

['Änderung des Bürgerlichen Gesetzbuchs\nDas Bürgerliche Gesetzbuch in der Fassung der Bekanntmachung vom 2. Januar 2002 (BGBl.',
 'Änderung des Einführungsgesetzes zum Bürgerlichen Gesetzbuche\nDem Artikel 229 des Einführungsgesetzes zum Bürgerlichen Gesetzbuche in der Fassung der']

In [63]:
#re.search('.+\n.+\n',re.split('Artikel [0-9]+  \n',text_of_change[6])[1]).captures()
#re.split('Artikel [0-9]+ \n',text_of_change[6])[1]
re.split('Artikel [0-9]+  \n',text_of_change[6])

['Drucksache 145/21\nEntwurf eines Gesetzes zum Ausbau des elektronischen Rechtsver-\nkehrs mit den Gerichten und zur Änderung weiterer prozessrechtlicher \nVorschriften1)  \nVom ... \nDer Bundestag hat das folgende Gesetz beschlossen: \nArtikel 1 \nÄnderung der Zivilprozessordnung \nDie Zivilprozessordnung in der Fassung der Bekanntmachung vom 5. Dezember 2005 \n(BGBl. I S. 3202; 2006 I S. 431; 2007 I S. 1781), die zuletzt durch Artikel 8 des Gesetzes \nvom 22. Dezember 2020 (BGBl. I S. 3320) geändert worden ist, wird wie folgt geändert: \n1. Die Inhaltsübersicht wird wie folgt geändert:\na) Der Angabe zu § 130a werden ein Semikolon und das Wort „Verordnungsermäch-\ntigung“ angefügt.\nb) Die Angaben zu den §§ 173 bis 176 werden wie folgt gefasst:\n„§ 173 Zustellung elektronischer Dokumente\n§174 Zustellung durch Aushändigung an der Amtsstelle\n§175 Zustellung eines Schriftstücks gegen Empfangsbekenntnis\n§176 Zustellung durch Einschreiben mit Rückschein; Zustellungsauftrag“.\n2. § 130

In [269]:
title_change_of_this_law = []
for page in range(first_page,last_page+1):
    has_artikel = len(text_of_change[page].split('Artikel '))
    if has_artikel >= 2:
        for artikel in range(1,has_artikel-1,2): # consider using re.split('Artikel [0-9]+ \n') instead
            title_change_of_this_law.append(text_of_change[page].split('Artikel ')[artikel].split(' \n')[1:3])
title_change_of_this_law

[['Änderung der Zivilprozessordnung',
  'Die Zivilprozessordnung in der Fassung der Bekanntmachung vom 5. Dezember 2005'],
 ['Änderung des Gesetzes über das Verfahren in Familiensachen',
  'und in Angelegenheiten der freiwilligen Gerichtsbarkeit'],
 ['Änderung der Elektronischer-Rechtsverkehr-Verordnung',
  'Die  Elektronischer-Rechtsverkehr-Verordnung  vom  24.  November  2017  (BGBl. I'],
 ['Änderung des Arbeitsgerichtsgesetzes',
  'Das Arbeitsgerichtsgesetz in der Fassung der Bekanntmachung vom 2. Juli 1979'],
 ['Weitere Änderung des Arbeitsgerichtsgesetzes',
  'Das Arbeitsgerichtsgesetz, das zuletzt durch '],
 ['Weitere Änderung des Arbeitsgerichtsgesetzes zum 1. Januar', '2022'],
 ['Weitere Änderung des Arbeitsgerichtsgesetzes zum 1. Januar', '2026'],
 ['Änderung des Sozialgerichtsgesetzes',
  'Das Sozialgerichtsgesetz in der Fassung der Bekanntmachung vom 23. September'],
 ['Weitere Änderung des Sozialgerichtsgesetzes zum 1. Januar 2022',
  'In § 65d Satz 2 des Sozialgerichtsgese

In [17]:
with pdfplumber.open(pdfFileObj) as pdf:
    for pageNum in pdf.pages()
	name_of_law_to_change = pdf.pages[6].extract_text().split('Artikel 1')[1].split('\n')[1]
    print(name_of_law_to_change)

Änderung der Zivilprozessordnung 


In [179]:
text_of_change[24].split('Artikel ')[1].split('\n')[1]

'Inkrafttreten '

In [167]:
len(text_of_change[6].split('Artikel '))
    
    

3

In [188]:
len(title_change_of_this_law[3])


2

In [73]:
text_of_change[4]

'Bundesrat  Drucksache   145/21  \n  \n  \n \n  12.02.21 \n  R - AIS - Fz - In \n \nB\nGesetzentwurf  R\nFu\nder Bundesregierung \nss \nEntwurf  eines  Gesetzes  zum  Ausbau  des  elektronischen \nRechtsverkehrs  mit  den  Gerichten  und  zur  Änderung  weiterer \nprozessrechtlicher Vorschriften \nBundesrepublik Deutschland             Berlin, 12. Februar 2021 \n      Die Bundeskanzlerin \n \nAn den \nPräsidenten des Bundesrates \nHerrn Ministerpräsidenten \nDr. Reiner Haseloff \n \nSehr geehrter Herr Präsident, \n \nhiermit übersende ich gemäß Artikel 76 Absatz 2 des Grundgesetzes den von der \nBundesregierung beschlossenen \n \nEntwurf eines Gesetzes zum Ausbau des elektronischen \nRechtsverkehrs mit den Gerichten und zur Änderung \nweiterer prozessrechtlicher Vorschriften  \n \nmit Begründung und Vorblatt. \n \nFederführend ist das Bundesministerium der Justiz und für Verbraucherschutz. \n \nDie  Stellungnahme  des  Nationalen  Normenkontrollrates  gemäß  § 6  Absatz 1 \nNKRG ist al

In [89]:
text_of_change[5] is None

True